In [1]:
import ee
import folium
import geehydro
import geemap

C:\Users\srini\AppData\Local\Programs\Python\Python310\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.11) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [2]:
ee.Initialize(project='ml-meets-the-sun')

In [3]:
Map = geemap.Map(center = (15.3173,75.7139), zoom =5)
Map.add_basemap('HYBRID')

In [4]:
Map

Map(center=[15.3173, 75.7139], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topri…

In [31]:
karnataka = ee.FeatureCollection('projects/ml-meets-the-sun/assets/Karnataka')
TamilNadu = ee.FeatureCollection('projects/ml-meets-the-sun/assets/TamilNadu')
Rajasthan = ee.FeatureCollection('projects/ml-meets-the-sun/assets/Rajasthan')

In [32]:
def map_display(state,name):
    vis_params = {
    'color': 'red'
    }
    Map.addLayer(state, vis_params, name, True, 0.5)
    return Map
Map

Map(bottom=55650.0, center=[26.926967642880527, 37.07061767578126], controls=(WidgetControl(options=['position…

In [7]:
states = {
    'Karnataka': karnataka,
    'Tamil Nadu': TamilNadu,
    'Rajasthan': Rajasthan
}
for name,state in states.items():
    map_display(state,name)

In [33]:
sent_cloud = (
    ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')
    .select('cs_cdf')
    .filter(ee.Filter.date('2020-01-01', '2025-07-31'))
)
display(sent_cloud)

In [34]:
sent_cloud.first()

In [51]:
mean_cloud_cover= sent_cloud.mean().setDefaultProjection(
    sent_cloud.first().projection()
)


In [38]:
mean_cloud_cover

In [39]:
mean_cloud_cover.projection()

In [52]:
result_clip = {}

def mean_cc_clip(state, name):
    cname = name + "_cc"
    clipped = mean_cloud_cover.clip(state)
    result_clip[cname] = clipped

In [53]:
for name,state in states.items():
    mean_cc_clip(state, name)

In [54]:
result_clip

{'Karnataka_cc': <ee.image.Image at 0x271b2b16ec0>,
 'Tamil Nadu_cc': <ee.image.Image at 0x271b2b14220>,
 'Rajasthan_cc': <ee.image.Image at 0x271b2b17490>}

In [55]:
def display_cc(cname, clipped):
    
    vis_rh ={
    'min': 0,
    'max': 100,
    'palette': [
        '000080',
        '0000d9',
        '4000ff',
        '8000ff',
        '0080ff',
        '00ffff',
        '00ff80',
        '80ff00',
        'daff00',
        'ffff00',
        'fff500',
        'ffda00',
        'ffb000',
        'ffa400',
        'ff4f00',
        'ff2500',
        'ff0a00',
        'ff00ff',
    ],
    }
    Map.addLayer(clipped,  vis_rh, cname, True, 0.5)
    return Map

In [56]:
for cname,clipped in result_clip.items():
    display_rh(cname, clipped)

In [63]:
sent_cloud.first().reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry= Rajasthan.geometry(),
    scale=10000,
    maxPixels=1e13
).getInfo()


{'cs_cdf_max': None, 'cs_cdf_min': None}

In [58]:
region = karnataka.geometry()

task =ee.batch.Export.image.toAsset(
    image=result_clip['Karnataka_cc'],
    description='KA_cc_mean',
    assetId='projects/ml-meets-the-sun/assets/KA_cc_mean',
    region=region,
    crsTransform = [0.1, 0, -180.05, 0, -0.1, 90.05],
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()

In [59]:
region = TamilNadu.geometry()

task =ee.batch.Export.image.toAsset(
    image=result_clip['Tamil Nadu_cc'],
    description='TN_cc_mean',
    assetId='projects/ml-meets-the-sun/assets/TN_cc_mean',
    region=region,
    crsTransform = [0.1, 0, -180.05, 0, -0.1, 90.05],
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()

In [60]:
region = Rajasthan.geometry()

task =ee.batch.Export.image.toAsset(
    image=result_clip['Rajasthan_cc'],
    description='RA_cc_mean',
    assetId='projects/ml-meets-the-sun/assets/RA_cc_mean',
    region=region,
    crsTransform = [0.1, 0, -180.05, 0, -0.1, 90.05],
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()